	- LLM Node
		- Prompt
			- Role: Expert Claims Consultant
			- describes how each tool can be used
			- Use tools to do research
			- Determine if the claim in valid via the policy
				- if yes write an email to the insurance company including the scenario and why it is a valid claim
				- if not let the client know why and what they can do to strengthen their claim
		- Access To Tools
	- Tools Node
		- RAG on Uploaded Policy
		- Web Search: 
			- Query what makes for a good claim
			- Explain the technical details in terms of what happened (ie. why wind pushed down a tree)
	- Use LangChain and LangGraph to build a simple agent, with an LLM node that does reasoning and termines what tools to call with what query and a tool node which takes the action

## Imports and Environment Variables

In [1]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API Key:")

In [2]:
os.environ["TAVILY_API_KEY"] = getpass.getpass("TAVILY_API_KEY")

In [3]:
os.environ["COHERE_API_KEY"] = getpass.getpass("Please enter your Cohere API key!")

In [4]:
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "ClaimAssist"
os.environ["LANGSMITH_API_KEY"] = getpass.getpass("Enter your Langsmith API Key:")

## Create Retriever for Uploaded Policy

In [5]:
from langchain_openai.embeddings import OpenAIEmbeddings

embedding_model = OpenAIEmbeddings(model="text-embedding-3-small")
embedding_dim = 1536

In [6]:
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams

insurance_policy_client = QdrantClient(":memory:")

insurance_policy_client.create_collection(
    collection_name="insurance_policy",
    vectors_config=VectorParams(size=embedding_dim, distance=Distance.COSINE),
)

insurance_policy_vector_store = QdrantVectorStore(
    client=insurance_policy_client,
    collection_name="insurance_policy",
    embedding=embedding_model,
)

In [7]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("./data/oklahoma_home_insurance_policy.pdf", mode="single")
insurance_policy_doc = loader.load()

print(insurance_policy_doc)

[Document(metadata={'producer': 'Acrobat Distiller 7.0.5 (Windows)', 'creator': 'Adobe PageMaker 7.0', 'creationdate': '2007-10-18T14:41:31-05:00', 'subject': 'HO3', 'author': 'WPD4481', 'keywords': '', 'moddate': '2007-10-18T14:41:35-04:00', 'title': 'HO3', 'source': './data/oklahoma_home_insurance_policy.pdf', 'total_pages': 29}, page_content="HO-4 (01-07)\nFor information regarding this policy, please\ncontact your Shelter Insurance Agent.\nHOMEOWNERS'\nINSURANCE\nPOLICY\nSHELTER INSURANCE COMPANIES\nHome Office: Columbia, MO 65218-0001\ndfg\nHO-4\n(01-07)\nTO OUR CUSTOMERS – PLEASE NOTE\nPlease read this policy carefully.  If you have\nquestions, contact your Shelter Agent for answers.\nNo agent can know your exact coverage needs or\nbudget considerations, so it is your responsibility to\nexamine the policy and make sure it provides the\ntypes of coverage you need in the amounts you\nrequested.\nIf you suffer a loss, please read this policy again so\nthat you will be reminded of yo

In [8]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
                    chunk_size=1000,
                    chunk_overlap=200
                )

insurance_policy_chunks = text_splitter.split_documents(insurance_policy_doc)
len(insurance_policy_chunks)

116

In [9]:
_ = insurance_policy_vector_store.add_documents(documents=insurance_policy_chunks)

In [10]:
insurance_policy_retriever = insurance_policy_vector_store.as_retriever(search_kwargs={"k": 5})

In [11]:
insurance_policy_retriever.invoke("What is the coverage for a fire?")

[Document(metadata={'producer': 'Acrobat Distiller 7.0.5 (Windows)', 'creator': 'Adobe PageMaker 7.0', 'creationdate': '2007-10-18T14:41:31-05:00', 'subject': 'HO3', 'author': 'WPD4481', 'keywords': '', 'moddate': '2007-10-18T14:41:35-04:00', 'title': 'HO3', 'source': './data/oklahoma_home_insurance_policy.pdf', 'total_pages': 29, '_id': 'c3faeaa2dc47438f846bb86f4f3b7ac2', '_collection_name': 'insurance_policy'}, page_content='No coverage is provided  under this section for\nthe removal of trees, shrubs, plants, and lawns.\nCoverage for those items, if any, is provided\nunder a specific heading elsewhere in this policy.\nNo coverage is provided under this section for\nthe fees or expenses you incur for tearing off,\nor tearing out, the remains of covered property.\nCoverage for those items, if any, is a part of the\nrestoration cost associated with the specific loss.\n2. Fire Department Charges\nWe will pay up to $500 for your liability under\nan agreement for service charges made by a

## BUILD RAG TOOLS

In [12]:
from typing import Annotated
from langchain_core.tools import tool

@tool
def retrieve_insurance_policy(
    query: Annotated[str, "query to ask the retrieve insurance policy tool"]
    ):
  """Use Retrieval Augmented Generation to retrieve information insurance policy clauses to determine if a claim is covered"""
  return insurance_policy_retriever.invoke(query)

In [13]:
from langchain_community.tools.tavily_search import TavilySearchResults

tavily_tool = TavilySearchResults(max_results=5)

/var/folders/r5/wsbp5_jj7lsfh6rxrw7p7lv00000gn/T/ipykernel_92811/1911882425.py:3: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-tavily package and should be used instead. To use it run `pip install -U :class:`~langchain-tavily` and import as `from :class:`~langchain_tavily import TavilySearch``.
  tavily_tool = TavilySearchResults(max_results=5)


In [14]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")
tool_belt = [retrieve_insurance_policy, tavily_tool]
llm_with_tools = llm.bind_tools(tool_belt)

In [15]:
from typing import TypedDict, Annotated
from langgraph.graph.message import add_messages

class AgentState(TypedDict):
  user_input: str
  messages: Annotated[list, add_messages]

In [16]:
from langchain_core.messages import HumanMessage, SystemMessage
system_prompt = """
You are a highly experienced Insurance Claim Consultant.

Your job is to evaluate whether a user's insurance claim is valid, based on the uploaded insurance policy. You have access to two tools:

1. **RAG Search on Insurance Policy** – Use this to search and retrieve relevant clauses from the user’s uploaded policy.
2. **Web Search Tool** – Use this to research any external facts (e.g., explanations of storm mechanics, typical standards for valid claims, or definitions of insurance terminology) to help clarify or strengthen your response.

---

📌 **Tool Usage Guidelines**

- **Always use the RAG Search first.** Your primary responsibility is to validate the claim strictly based on the user’s uploaded policy. Use this tool as often as needed to understand the policy and how it applies to the user's claim.
- **Only use the Web Search if needed**, after the RAG Search:
  - If the policy is unclear or lacks specific wording on the situation described
  - If the user mentions a complex or technical situation that may require external explanation (e.g., “derecho wind event”, “burst pipe due to negative pressure”)
- **Do not overuse the Web Search.** Your main objective is to help the user understand whether their claim is supported by the actual policy document.

---

🧠 **Agent Behavior**

1. **Parse the user’s claim** — Identify the scenario, cause of loss, location, date, and keywords (e.g., water damage, wind, theft).
2. **Query the Insurance Policy using the RAG Search tool** to find clauses that mention covered perils, exclusions, and conditions.
3. **Determine if the claim is valid or not.** Base your decision primarily on what the policy says.
4. If necessary, **use the Web Search tool** to clarify any uncertain facts or explain why a technical detail supports or weakens the claim.
5. **Generate a response to the user:**
   - If the claim is valid: write a professional email the user can send to their insurance company, referencing relevant clauses.
   - If the claim is not valid: explain why not, and give clear, actionable suggestions on what the user can do to strengthen or reframe the claim.

---

🔒 Always ground your decision in the uploaded policy first, and be concise, helpful, and accurate.
"""

def prepare_input(state):
  messages = []
  messages.append(SystemMessage(content=system_prompt))
  messages.append(HumanMessage(content=state["user_input"]))
  return {"messages": messages}


In [17]:
from langgraph.prebuilt import ToolNode

def call_model(state):
  messages = state["messages"]
  response = llm_with_tools.invoke(messages)
  return {"messages" : [response]}

tool_node = ToolNode(tool_belt)

## Build The Graph

In [18]:
from langgraph.graph import StateGraph, END

uncompiled_graph = StateGraph(AgentState)

uncompiled_graph.add_node("prepare_input", prepare_input)
uncompiled_graph.add_node("agent", call_model)
uncompiled_graph.add_node("action", tool_node)
uncompiled_graph.set_entry_point("prepare_input")

def should_continue(state):
  last_message = state["messages"][-1]

  if last_message.tool_calls:
    return "action"

  return END

uncompiled_graph.add_conditional_edges(
    "agent",
    should_continue
)

uncompiled_graph.add_edge("prepare_input", "agent")
uncompiled_graph.add_edge("action", "agent")


claim_consultant_agent = uncompiled_graph.compile()


## Testing Initial Solution

In [19]:
query = """
A sudden freeze on January 17th caused a pipe in our upstairs bathroom to burst overnight. We woke up to water leaking through the ceiling into the kitchen. We immediately shut off the water and contacted a plumber. Since the damage was accidental and not due to neglect, we are submitting a claim for the cost of plumbing repair and water mitigation services.
"""

input = {"user_input": query}
async for chunk in claim_consultant_agent.astream(input, stream_mode="updates"):
    for node, values in chunk.items():
        print(f"Receiving update from node: '{node}'")
        print(values["messages"])
        print("\n\n")

Receiving update from node: 'prepare_input'
[SystemMessage(content="\nYou are a highly experienced Insurance Claim Consultant.\n\nYour job is to evaluate whether a user's insurance claim is valid, based on the uploaded insurance policy. You have access to two tools:\n\n1. **RAG Search on Insurance Policy** – Use this to search and retrieve relevant clauses from the user’s uploaded policy.\n2. **Web Search Tool** – Use this to research any external facts (e.g., explanations of storm mechanics, typical standards for valid claims, or definitions of insurance terminology) to help clarify or strengthen your response.\n\n---\n\n📌 **Tool Usage Guidelines**\n\n- **Always use the RAG Search first.** Your primary responsibility is to validate the claim strictly based on the user’s uploaded policy. Use this tool as often as needed to understand the policy and how it applies to the user's claim.\n- **Only use the Web Search if needed**, after the RAG Search:\n  - If the policy is unclear or lacks 

## Strategy Experimentation

### Visualizing the Chunks

#### Naive Chunking

In [30]:
import textwrap
def visualize_chunks(chunks, start=0, end=10):
  for chunk in chunks[start:end]:
    print("="*120)
    print(textwrap.fill(chunk.page_content, width=150)) 

In [31]:
visualize_chunks(insurance_policy_chunks, 10, 20)

relative, if the individual engaged in that activity is under the age of twenty-five, and is a full time student. 6. Claim  means a request by any
person  for benefits under this policy as a result of any one accident. It includes lawsuits, requests for the payment of money and requests that we
take any action, or extend any coverage, provided for by this policy. 7. Compensation Law means any law under which benefits are paid to a person as
compensation for the effects of bodily injury, without regard to fault, because of that person’s status as an employee or beneficiary. It includes,
but is not limited to, workers’ compensation laws, disability laws, the Federal Employers’ Liability Act and the Jones Act. 8. Custom farming means
the maintenance or use of premises, other than insured premises, for the production of farm products and includes all operations reasonably necessary
for such production that are performed: (a) at the request of the owner or renter of those premises; and
the 

### Parent Document Retrieval Strategy

In [40]:
from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import InMemoryStore
from langchain_text_splitters import RecursiveCharacterTextSplitter
from qdrant_client import QdrantClient, models

parent_docs = insurance_policy_doc

parent_splitter = child_splitter = RecursiveCharacterTextSplitter(chunk_size=3500, chunk_overlap=400)
child_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

In [41]:
insurance_policy_client.create_collection(
    collection_name="full_documents",
    vectors_config=models.VectorParams(size=1536, distance=models.Distance.COSINE)
)

parent_document_vectorstore = QdrantVectorStore(
    collection_name="full_documents", embedding=OpenAIEmbeddings(model="text-embedding-3-small"), client=insurance_policy_client
)

ValueError: Collection full_documents already exists

In [46]:
store = InMemoryStore()
parent_document_retriever = ParentDocumentRetriever(
    vectorstore=parent_document_vectorstore,
    docstore=store,
    child_splitter=child_splitter,
    parent_splitter=parent_splitter,
    store=store,
    k=3
)

In [47]:
parent_document_retriever.add_documents(parent_docs, ids=None)

In [48]:
retrieved_docs = parent_document_retriever.invoke("What is the coverage for a burst pipe?")

visualize_chunks(retrieved_docs)

pump system, or by an obstruction of such that is located off of the residence premises. (d) Water, or water borne contaminants or materials, below
the surface of the ground, that exerts pressure on, or flows, seeps, or leaks, through any part of a building or other structure, sidewalk, driveway
or pool. (e) Condensation of water vapor. We do cover accidental direct physical loss that occurs subsequent to any of the events or conditions listed
in 3(a), 3(b), 3(c), 3(d) and 3(e), above, if that loss is caused by theft, fire or explosion. 4. Power, heating, or cooling failure or interruption,
unless it results from accidental direct physical loss to power, heating or cooling equipment located on the residence premises and that loss is
caused by a peril we insure against. We do cover accidental direct physical loss that occurs subsequent to any of these events if that loss is caused
by theft, fire, or explosion. 14  HO-4 (01-07) 5. Neglect of an insured  to use all reasonable means to pro

In [49]:
from typing import Annotated
from langchain_core.tools import tool

@tool
def retrieve_insurance_policy_parent_document(
    query: Annotated[str, "query to ask the retrieve insurance policy tool"]
    ):
  """Use Retrieval Augmented Generation to retrieve information insurance policy clauses to determine if a claim is covered"""
  return parent_document_retriever.invoke(query)

In [50]:
llm_parent_document = ChatOpenAI(model="gpt-4o-mini")
tool_belt_parent_document = [retrieve_insurance_policy_parent_document, tavily_tool]
llm_with_tools_parent_document = llm_parent_document.bind_tools(tool_belt_parent_document)
tool_node_parent_document = ToolNode(tool_belt_parent_document)

In [51]:
from langgraph.prebuilt import ToolNode

def call_model_parent_document(state):
  messages = state["messages"]
  response = llm_with_tools_parent_document.invoke(messages)
  return {"messages" : [response]}

tool_node_parent_document = ToolNode(tool_belt_parent_document)

In [52]:
from langgraph.graph import StateGraph, END

uncompiled_parent_document_graph = StateGraph(AgentState)

uncompiled_parent_document_graph.add_node("prepare_input", prepare_input)
uncompiled_parent_document_graph.add_node("agent", call_model_parent_document)
uncompiled_parent_document_graph.add_node("action", tool_node_parent_document)
uncompiled_parent_document_graph.set_entry_point("prepare_input")

uncompiled_parent_document_graph.add_conditional_edges(
    "agent",
    should_continue
)

uncompiled_parent_document_graph.add_edge("prepare_input", "agent")
uncompiled_parent_document_graph.add_edge("action", "agent")

parent_document_claim_consultant_agent = uncompiled_parent_document_graph.compile()

In [53]:
query = """
A sudden freeze on January 17th caused a pipe in our upstairs bathroom to burst overnight. We woke up to water leaking through the ceiling into the kitchen. We immediately shut off the water and contacted a plumber. Since the damage was accidental and not due to neglect, we are submitting a claim for the cost of plumbing repair and water mitigation services.
"""

input = {"user_input": query}
async for chunk in parent_document_claim_consultant_agent.astream(input, stream_mode="updates"):
    for node, values in chunk.items():
        print(f"Receiving update from node: '{node}'")
        print(values["messages"])
        print("\n\n")

Receiving update from node: 'prepare_input'
[SystemMessage(content="\nYou are a highly experienced Insurance Claim Consultant.\n\nYour job is to evaluate whether a user's insurance claim is valid, based on the uploaded insurance policy. You have access to two tools:\n\n1. **RAG Search on Insurance Policy** – Use this to search and retrieve relevant clauses from the user’s uploaded policy.\n2. **Web Search Tool** – Use this to research any external facts (e.g., explanations of storm mechanics, typical standards for valid claims, or definitions of insurance terminology) to help clarify or strengthen your response.\n\n---\n\n📌 **Tool Usage Guidelines**\n\n- **Always use the RAG Search first.** Your primary responsibility is to validate the claim strictly based on the user’s uploaded policy. Use this tool as often as needed to understand the policy and how it applies to the user's claim.\n- **Only use the Web Search if needed**, after the RAG Search:\n  - If the policy is unclear or lacks 

### Contextual Compression Retrieval

In [54]:
adjusted_insurance_policy_retriever = insurance_policy_vector_store.as_retriever(search_kwargs={"k": 20})

In [55]:
from langchain.retrievers.document_compressors import FlashrankRerank
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever

# FlashRank - works offline, no API key needed
compressor = FlashrankRerank()

# Create compression retriever with your base retriever
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=adjusted_insurance_policy_retriever  # Your existing retriever
)

In [56]:
from typing import Annotated
from langchain_core.tools import tool

@tool
def retrieve_insurance_policy_advanced(
    query: Annotated[str, "query to ask the retrieve insurance policy tool"]
    ):
  """Use Retrieval Augmented Generation to retrieve information insurance policy clauses to determine if a claim is covered"""
  return compression_retriever.invoke(query)

In [57]:
llm_advanced = ChatOpenAI(model="gpt-4o-mini")
tool_belt_advanced = [retrieve_insurance_policy_advanced, tavily_tool]
llm_with_tools_advanced = llm_advanced.bind_tools(tool_belt_advanced)
tool_node_advanced = ToolNode(tool_belt_advanced)

In [58]:
from langgraph.prebuilt import ToolNode

def call_model_advanced(state):
  messages = state["messages"]
  response = llm_with_tools_advanced.invoke(messages)
  return {"messages" : [response]}

tool_node_advanced = ToolNode(tool_belt_advanced)

In [59]:
from langgraph.graph import StateGraph, END

uncompiled_advanced_graph = StateGraph(AgentState)

uncompiled_advanced_graph.add_node("prepare_input", prepare_input)
uncompiled_advanced_graph.add_node("agent", call_model_advanced)
uncompiled_advanced_graph.add_node("action", tool_node_advanced)
uncompiled_advanced_graph.set_entry_point("prepare_input")

uncompiled_advanced_graph.add_conditional_edges(
    "agent",
    should_continue
)

uncompiled_advanced_graph.add_edge("prepare_input", "agent")
uncompiled_advanced_graph.add_edge("action", "agent")

advanced_claim_consultant_agent = uncompiled_advanced_graph.compile()

In [60]:
query = """
A sudden freeze on January 17th caused a pipe in our upstairs bathroom to burst overnight. We woke up to water leaking through the ceiling into the kitchen. We immediately shut off the water and contacted a plumber. Since the damage was accidental and not due to neglect, we are submitting a claim for the cost of plumbing repair and water mitigation services.
"""

input = {"user_input": query}
async for chunk in advanced_claim_consultant_agent.astream(input, stream_mode="updates"):
    for node, values in chunk.items():
        print(f"Receiving update from node: '{node}'")
        print(values["messages"])
        print("\n\n")

Receiving update from node: 'prepare_input'
[SystemMessage(content="\nYou are a highly experienced Insurance Claim Consultant.\n\nYour job is to evaluate whether a user's insurance claim is valid, based on the uploaded insurance policy. You have access to two tools:\n\n1. **RAG Search on Insurance Policy** – Use this to search and retrieve relevant clauses from the user’s uploaded policy.\n2. **Web Search Tool** – Use this to research any external facts (e.g., explanations of storm mechanics, typical standards for valid claims, or definitions of insurance terminology) to help clarify or strengthen your response.\n\n---\n\n📌 **Tool Usage Guidelines**\n\n- **Always use the RAG Search first.** Your primary responsibility is to validate the claim strictly based on the user’s uploaded policy. Use this tool as often as needed to understand the policy and how it applies to the user's claim.\n- **Only use the Web Search if needed**, after the RAG Search:\n  - If the policy is unclear or lacks 

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Receiving update from node: 'agent'
[AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_wvYABqpGEqgBUSENw4J0E19w', 'function': {'arguments': '{"query":"burst pipe, sudden freeze, water damage, coverage for plumbing repair and water mitigation"}', 'name': 'retrieve_insurance_policy_advanced'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 33, 'prompt_tokens': 673, 'total_tokens': 706, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-C10SWtYALQNd9tZrYQ1M7dVLACmN7', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--5bcd4ea7-4518-4122-a064-662cf4cc62ea-0', tool_calls=[{'name': 'retrieve_insurance_policy_advanced', 'args': {'query': 'burst p

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


Receiving update from node: 'action'
[ToolMessage(content="[Document(metadata={'id': 8, 'relevance_score': np.float32(0.9991507), 'producer': 'Acrobat Distiller 7.0.5 (Windows)', 'creator': 'Adobe PageMaker 7.0', 'creationdate': '2007-10-18T14:41:31-05:00', 'subject': 'HO3', 'author': 'WPD4481', 'keywords': '', 'moddate': '2007-10-18T14:41:35-04:00', 'title': 'HO3', 'source': './data/oklahoma_home_insurance_policy.pdf', 'total_pages': 29, '_id': 'ab1991377f5d4fb487f7641730fddf38', '_collection_name': 'insurance_policy'}, page_content='loss to power, heating or cooling equipment\\nlocated on the residence premises and that loss\\nis caused by a peril we insure against. We do\\ncover accidental direct physical loss that occurs\\nsubsequent to any of these events if that loss is\\ncaused by theft, fire, or explosion.\\n14\\n\\x0cHO-4 (01-07)\\n5. Neglect of an insured  to use all reasonable\\nmeans to protect covered property at and after\\nthe time of loss or when property is threatened\

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Receiving update from node: 'agent'
[AIMessage(content='Based on the information you provided and the relevant clauses from your insurance policy, it looks like your claim may be valid.\n\n### Key Points from Your Policy:\n1. **Covered Perils**: The policy appears to cover “accidental direct physical loss” caused by certain events. While the specific wording related to burst pipes due to freezing was not retrieved, often home insurance policies do cover water damage resulting from burst pipes unless specifically excluded.\n2. **Neglect Clause**: Your situation indicates immediate action was taken (shutting off the water and contacting a plumber), which typically demonstrates that you used reasonable means to protect your property, likely complying with the policy\'s terms regarding neglect.\n\n### Conclusion:\nGiven this information, your claim for the cost of plumbing repair and water mitigation services caused by the burst pipe due to the sudden freeze should be supported by your ins

## Generate Golden Dataset using RAGAS

In [61]:
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings

generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4.1-nano"))
generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings())

/Users/terellbrown/terellcodes/claim-mate/api/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# Create custom query distribution focusing on claim scenarios
from ragas.testset.synthesizers import MultiHopAbstractQuerySynthesizer, MultiHopSpecificQuerySynthesizer

# Custom query distribution - prioritize single-hop claim scenarios
claim_query_distribution = [
    (claim_synthesizer, 0.7),  # 70% single-hop claim scenarios
    (MultiHopAbstractQuerySynthesizer(llm=generator_llm), 0.15),  # 15% multi-hop abstract
    (MultiHopSpecificQuerySynthesizer(llm=generator_llm), 0.15),  # 15% multi-hop specific
]

# Generate dataset using the custom distribution
from ragas.testset import TestsetGenerator

claim_generator = TestsetGenerator(
    llm=generator_llm, 
    embedding_model=generator_embeddings
)

# Generate claim dataset
claim_dataset = claim_generator.generate_with_langchain_docs(
    insurance_policy_chunks[:10],  # Use more chunks for variety
    testset_size=15,
    query_distribution=claim_query_distribution
)

In [ ]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings)
dataset = generator.generate_with_langchain_docs(insurance_policy_chunks[:2], testset_size=10)

## Generate Golden Dataset using RAGAS

### Create Custom Query Synthesizers

#### Custom Single Hop Query Synthesizers

In [66]:
import typing as t

from pydantic import BaseModel

from ragas.prompt import PydanticPrompt
from ragas.testset.persona import Persona

from ragas.testset.synthesizers.single_hop.prompts import QueryCondition, GeneratedQueryAnswer


class CustomQueryAnswerGenerationPrompt(PydanticPrompt[QueryCondition, GeneratedQueryAnswer]):
    instruction: str = (
        "Generate a realistic insurance claim description and determine its validity based on the provided policy context. "
        "The claim should sound like a real homeowner describing an incident to their insurance company, and the response "
        "should evaluate if the claim is covered based solely on the policy terms."
        "### Instructions:\n"
        "1. **Generate a Claim**: Based on the context, persona, term, style, and length, create a realistic insurance claim "
        "that describes an incident related to the policy coverage or exclusions. Include specific details like dates and damages.\n"
        "2. **Generate a Response**: Using only the content from the provided context, determine whether the claim would be "
        "covered under the policy. Cite specific policy language to justify the response.\n"
    )
    input_model: t.Type[QueryCondition] = QueryCondition 
    output_model: t.Type[GeneratedQueryAnswer] = GeneratedQueryAnswer
    examples: t.List[t.Tuple[QueryCondition, GeneratedQueryAnswer]] = [
        (
            QueryCondition(
                persona=Persona(
                    name="Homeowner",
                    role_description="A policyholder filing an insurance claim.",
                ),
                term="water damage",
                query_style="Formal",
                query_length="Medium", 
                context="We do not cover loss caused by continuous or repeated seepage or leakage of water or steam from a plumbing system over a period of weeks, months or years. We do cover sudden and accidental discharge or overflow of water or steam from a plumbing system.",
            ),
            GeneratedQueryAnswer(
                query="On March 15th, I discovered significant water damage in my kitchen from a burst pipe under the sink. The pipe suddenly burst while I was at work, flooding the kitchen floor and damaging the cabinets. I shut off the water immediately when I got home and called a plumber for emergency repairs.",
                answer="Based on the policy terms, this claim would be covered as it involves a sudden and accidental discharge of water from a plumbing system. The policy specifically covers 'sudden and accidental discharge or overflow of water or steam from a plumbing system.' This is distinct from gradual water damage that occurs over weeks or months, which would be excluded.",
            ),
        ),
    ]


In [67]:
from ragas.testset.synthesizers import SingleHopSpecificQuerySynthesizer
synthesizer = SingleHopSpecificQuerySynthesizer(llm=generator_llm)

synthesizer.generate_query_reference_prompt = CustomQueryAnswerGenerationPrompt()

print(query)


A sudden freeze on January 17th caused a pipe in our upstairs bathroom to burst overnight. We woke up to water leaking through the ceiling into the kitchen. We immediately shut off the water and contacted a plumber. Since the damage was accidental and not due to neglect, we are submitting a claim for the cost of plumbing repair and water mitigation services.



In [ ]:
query.instruction = """
You are generating a **realistic insurance claim description**, not a question.

Each claim should:
- Be 3–5 sentences long
- Sound like a real homeowner trying to describe an incident to their insurance company
- Mention a date or timeframe
- Be directly related to the content of the provided context (e.g., perils, exclusions, coverage)
- Avoid asking questions — this is not a Q&A task

Only return a single claim scenario per output. Begin immediately with the claim. Do not include headers like "Claim:" or "Scenario:".
"""

In [ ]:
import pandas as pd
pd.set_option('display.max_colwidth', None)
claim_dataset.to_pandas()

,user_input,reference_contexts,reference,synthesizer_name
0,"I recently moved to Columbia and noticed some water stains on my ceiling after a heavy rainstorm on April 10th. Upon inspection, I found that the roof had a leak, which caused water to seep into my attic and damage some of my stored belongings. I had a roofing contractor repair the leak on April 12th. I am requesting coverage for the damages caused by this leak.","[HO-4 (01-07)\nFor information regarding this policy, please\ncontact your Shelter Insurance Agent.\nHOMEOWNERS'\nINSURANCE\nPOLICY\nSHELTER INSURANCE COMPANIES\nHome Office: Columbia, MO 65218-0001\ndfg\nHO-4\n(01-07)\nTO OUR CUSTOMERS – PLEASE NOTE\nPlease read this policy carefully. If you have\nquestions, contact your Shelter Agent for answers.\nNo agent can know your exact coverage needs or\nbudget considerations, so it is your responsibility to\nexamine the policy and make sure it provides the\ntypes of coverage you need in the amounts you\nrequested.\nIf you suffer a loss, please read this policy again so\nthat you will be reminded of your rights and\nobligations. It is very important for you to recognize\nthat this insurance policy is a legally binding\ncontract. If any insured fails to perform an\nobligation required by this policy, the coverage\nwhich it might otherwise provide could be lost.\n HO-4 (01-07)\nTHE INDEX\nWHERE YOU CAN FIND IT]","Based on the policy terms, this claim may not be covered. The policy does not specify coverage for roof leaks or water damage resulting from weather-related causes. Since the damage was caused by a leak during a storm, and the policy emphasizes the importance of reviewing coverage details with your Shelter Agent, it is likely that this type of damage is excluded unless explicitly covered. Therefore, without confirmation of coverage for roof leaks caused by weather, this claim would probably not be eligible for reimbursement under the current policy.",single_hop_specifc_query_synthesizer
1,"On April 10th, I accidentally damaged my roof during a storm, causing a leak that led to water seeping into my attic and damaging some personal belongings. I believe this damage was caused by the storm, which was sudden and unexpected. I am requesting coverage for repairs and replacement of damaged items.","[contract. If any insured fails to perform an\nobligation required by this policy, the coverage\nwhich it might otherwise provide could be lost.\n HO-4 (01-07)\nTHE INDEX\nWHERE YOU CAN FIND IT\nDeclarations - Your Name, Location of Your Residence, Policy Period, Limits of Liability and Deductibles\nBeginning on Page\nAgreement ............................................................................................................................................................. 2\nDefinitions ............................................................................................................................................................. 2\nGeneral Agreements applicable to entire policy .................................................................................................... 8\nSection I - Property Protection ........................................................................................................................... 10]","Based on the policy terms, this claim may not be covered. The policy states that if any insured fails to perform an obligation required by the policy, the coverage could be lost. Since the claim involves damage caused by a storm, which is typically considered an insured peril, coverage could be applicable. However, the policy also emphasizes the importance of adhering to obligations and proper maintenance. If the damage resulted from neglect or failure to perform required obligations, the claim might be denied. Without evidence that all obligations were met, the validity of this claim cannot be confirmed solely based on the provided policy language.",single_hop_specifc_query_synthesizer


#### Custom Multi Hop Query Synthesizer

In [79]:
agent_prompt = """
You are a highly experienced Insurance Claim Consultant.

Your job is to evaluate whether a user's insurance claim is valid, based on the context retrieved. 

1. Use retrieved context to find clauses that mention covered perils, exclusions, and conditions.
2. **Determine if the claim is valid or not.** Base your decision primarily on what the policy says.
3. **Return your response in JSON format with these exact fields:**

```json
{{
  "is_valid": true/false,
  "evaluation": "Detailed explanation of your analysis and reasoning. Use bullet points.",
  "citations": "List of citations from the policy that support your analysis",
  "email_draft": "Professional email to send to insurance company (only if is_valid is true)",
  "suggestions": "Actionable suggestions for the user (especially if is_valid is false)"
}}
```

**IMPORTANT**: You must ALWAYS return a valid JSON response with these exact field names. 
- Set "is_valid" to true only if you are confident the claim should be covered based on the policy
- Set "is_valid" to false if the claim is excluded, not covered, or you have significant doubts
- Always provide a detailed "evaluation" explaining your reasoning
- Only include "email_draft" if is_valid is true
- Always provide helpful "suggestions" for the user

Always ground your decision in the uploaded policy first, and be concise, helpful, and accurate.
"""

In [122]:
import typing as t
from ragas.testset.synthesizers.single_hop.prompts import QueryCondition, GeneratedQueryAnswer


class CustomQueryAnswerGenerationPrompt(PydanticPrompt[QueryCondition, GeneratedQueryAnswer]):
    instruction: str = (
        "Generate a realistic insurance claim description and determine its validity based on the provided policy context. "
        "The claim should sound like a real homeowner describing an incident to their insurance company, and the response "
        "should evaluate if the claim is covered based solely on the policy terms."
        "Some claims should be valid, some should be invalid."
        "### Instructions:\n"
        "1. **Generate a Claim**: Based on the context, persona, term, style, and length, create a realistic insurance claim "
        "that describes an incident related to the policy coverage or exclusions. Include specific details like dates and damages. Some claims should be valid, some should be invalid.\n"
        "2. **Generate a Response**: Using only the content from the provided context, determine whether the claim would be "
        "covered under the policy. Cite specific policy language to justify the response. If not covered, explain why.\n"
        "At the beginning of the response, state whether the claim is valid or invalid. Afterwards give an explanation why. If it is invalid, give suggestions for how to make it valid.\n"
    )
    input_model: t.Type[QueryCondition] = QueryCondition 
    output_model: t.Type[GeneratedQueryAnswer] = GeneratedQueryAnswer
    examples: t.List[t.Tuple[QueryCondition, GeneratedQueryAnswer]] = [
        (
            QueryCondition(
                persona=Persona(
                    name="Homeowner",
                    role_description="A policyholder filing an insurance claim.",
                ),
                term="water damage",
                query_style="Formal",
                query_length="Medium", 
                context="We do not cover loss caused by continuous or repeated seepage or leakage of water or steam from a plumbing system over a period of weeks, months or years. We do cover sudden and accidental discharge or overflow of water or steam from a plumbing system.",
            ),
            GeneratedQueryAnswer(
                query="On March 15th, I discovered significant water damage in my kitchen from a burst pipe under the sink. The pipe suddenly burst while I was at work, flooding the kitchen floor and damaging the cabinets. I shut off the water immediately when I got home and called a plumber for emergency repairs.",
                answer="Based on the policy terms, this claim would be covered as it involves a sudden and accidental discharge of water from a plumbing system. The policy specifically covers 'sudden and accidental discharge or overflow of water or steam from a plumbing system.' This is distinct from gradual water damage that occurs over weeks or months, which would be excluded.",
            ),
        ),
    ]

custom_single_hop_synthesizer = SingleHopSpecificQuerySynthesizer(llm=generator_llm)

custom_single_hop_synthesizer.generate_query_reference_prompt = CustomQueryAnswerGenerationPrompt()


In [123]:
import typing as t
from ragas.testset.synthesizers.multi_hop.prompts import QueryConditions, GeneratedQueryAnswer
from ragas.testset.synthesizers import MultiHopAbstractQuerySynthesizer, MultiHopSpecificQuerySynthesizer

class MultiHopCustomQueryAnswerGenerationPrompt(
    PydanticPrompt[QueryConditions, GeneratedQueryAnswer]
):
    instruction: str = (
        "Generate a complex insurance claim and determine its validity based on multiple sections of the insurance policy. "
        "The claim should require connecting information from different parts of the policy to determine coverage. "
        "The themes represent key policy concepts that should be incorporated into the claim scenario."
        "Some claims should be valid, some should be invalid."
        "### Instructions:\n"
        "1. **Generate a Multi-Part Claim**: Create a detailed insurance claim that involves multiple aspects of coverage "
        "from different policy sections (marked as `<1-hop>`, `<2-hop>`, etc.). The claim should realistically describe "
        "an incident that requires analyzing multiple policy provisions. Some claims should be valid, some should be invalid.\n"
        "2. **Generate a Coverage Analysis**: Using only the provided policy sections, determine whether the claim would be "
        "covered. Reference specific language from multiple sections to justify the conclusion. At the beggining of the response, state whether the claim is valid or invalid. After words give an explanation why. If it is invalid, give suggestions for how to make it valid.\n"
        "3. **Multi-Hop Context Tags**:\n"
        "   - Each policy section is tagged as `<1-hop>`, `<2-hop>`, etc.\n"
        "   - The claim and analysis must meaningfully connect provisions from multiple sections."
    )
    input_model: t.Type[QueryConditions] = QueryConditions
    output_model: t.Type[GeneratedQueryAnswer] = GeneratedQueryAnswer
    examples: t.List[t.Tuple[QueryConditions, GeneratedQueryAnswer]] = [
        (
            QueryConditions(
                persona=Persona(
                    name="Homeowner",
                    role_description="A policyholder filing a complex insurance claim.",
                ),
                themes=["Water Damage", "Vehicle Coverage"],
                query_style="Formal",
                query_length="Medium",
                context=[
                    "<1-hop> We do not cover loss caused by continuous or repeated seepage of water over weeks or months. We do cover sudden and accidental discharge of water.",
                    "<2-hop> We do not cover vehicles that are or were motorized, except those used primarily to service the residence premises.",
                ],
            ),
            GeneratedQueryAnswer(
                query="Last week during a severe storm, my riding lawnmower was damaged when my basement flooded suddenly due to a backed-up drain. The water rose quickly and submerged the mower before I could move it. The mower was stored in the basement because I use it weekly to maintain my property. Both the flooding and vehicle damage occurred within a few hours.",
                answer=(
                    "This claim requires analyzing both water damage and vehicle coverage provisions. The water damage would be covered since it was a 'sudden and accidental discharge' rather than gradual seepage. Additionally, while motorized vehicles are generally excluded, the riding lawnmower would be covered because it falls under the exception for vehicles 'used primarily to service the residence premises.' Therefore, based on both policy sections, this claim would likely be approved for both the water damage and mower damage."
                ),
            ),
        ),
    ]

custom_multi_hop_abstract_synthesizer = MultiHopAbstractQuerySynthesizer(llm=generator_llm)
custom_multi_hop_abstract_synthesizer.generate_query_reference_prompt = MultiHopCustomQueryAnswerGenerationPrompt()

custom_multi_hop_specific_synthesizer = MultiHopSpecificQuerySynthesizer(llm=generator_llm)
custom_multi_hop_specific_synthesizer.generate_query_reference_prompt = MultiHopCustomQueryAnswerGenerationPrompt()

### Dataset Generation

In [97]:
from ragas.testset.persona import Persona

# Persona 1: First-Time Homeowner
persona_first_time_homeowner = Persona(
    name="First-Time Homeowner",
    role_description="Recently bought a home and wants to file a claim for the first time. Is unsure about what types of events are covered and unfamiliar with policy exclusions."
)

# Persona 2: Storm Victim
persona_storm_victim = Persona(
    name="Storm Victim",
    role_description="Experienced a severe storm and now wants to file a claim for damages such as a fallen tree or roof leaks."
)

# Persona 3: Diligent Planner
persona_diligent_planner = Persona(
    name="Diligent Planner",
    role_description="Wants to file a claim in a thorough and informed way, carefully reviewing the policy and anticipating potential denial reasons."
)

# Persona 4: Elderly Retiree
persona_elderly_retiree = Persona(
    name="Elderly Retiree",
    role_description="Has lived in the same home for decades and wants to file a claim for what they believe is covered damage, though the issue may have developed over time."
)

# Persona 5: Frustrated Policyholder
persona_frustrated_policyholder = Persona(
    name="Frustrated Policyholder",
    role_description="Has had claims denied in the past and is now filing a new claim, determined to get clear justification and fair treatment from the insurance provider."
)

# Persona 6: Busy Professional
persona_busy_professional = Persona(
    name="Busy Professional",
    role_description="Has limited time to read policy details and wants to quickly file a claim for recent damage, hoping it is covered."
)

personas = [
    persona_first_time_homeowner,
    persona_storm_victim,
    persona_diligent_planner,
    persona_elderly_retiree,
    persona_frustrated_policyholder,
    persona_busy_professional,
]

In [121]:
from ragas.testset import TestsetGenerator

claim_generator = TestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings, persona_list=personas)

claim_query_distribution = [
    (custom_single_hop_synthesizer, 0.25),
    (custom_multi_hop_abstract_synthesizer, 0.375),
    (custom_multi_hop_specific_synthesizer, 0.375),
]

# Generate claim dataset
claim_dataset = claim_generator.generate_with_langchain_docs(
    insurance_policy_chunks[10:40],  # Use more chunks for variety
    testset_size=10,
    query_distribution=claim_query_distribution,
)

Applying SummaryExtractor:   0%|          | 0/30 [00:00<?, ?it/s]INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Applying SummaryExtractor:   3%|▎         | 1/30 [00:01<00:41,  1.43s/it]INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Applying SummaryExtractor:  27%|██▋       | 8/30 [00:01<00:03,  7.00it/s]INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/

In [351]:
# Create a deep copy of the claim dataset
import copy
claim_dataset_advanced = copy.deepcopy(claim_dataset)

In [124]:
import pandas as pd
pd.set_option('display.max_colwidth', None)
claim_dataset.to_pandas()

,user_input,reference_contexts,reference,synthesizer_name
0,"On April 10th, I was injured while working on my boat at the marina, which I believe is covered under the Jones Act. I sustained a back injury after lifting heavy equipment, and I am now unable to work. I am requesting benefits for my injury.","[relative, if the individual engaged in that\nactivity is under the age of twenty-five, and\nis a full time student.\n6. Claim means a request by any person for\nbenefits under this policy as a result of any one\naccident. It includes lawsuits, requests for the\npayment of money and requests that we take any\naction, or extend any coverage, provided for by\nthis policy.\n7. Compensation Law means any law under which\nbenefits are paid to a person as compensation\nfor the effects of bodily injury, without regard\nto fault, because of that person’s status as an\nemployee or beneficiary. It includes, but is not\nlimited to, workers’ compensation laws, disability\nlaws, the Federal Employers’ Liability Act and the\nJones Act.\n8. Custom farming means the maintenance or use\nof premises, other than insured premises, for\nthe production of farm products and includes all\noperations reasonably necessary for such\nproduction that are performed:\n(a) at the request of the owner or renter of\nthose premises; and]","This claim is invalid. The Jones Act applies specifically to benefits paid to individuals as a result of bodily injury incurred while working on a vessel or in maritime employment, typically related to the operation of ships or boats. However, the policy context provided indicates that the Jones Act pertains to compensation laws for bodily injuries related to employment, not personal injuries sustained outside of such employment or unrelated activities. Since the injury occurred while working at a marina on a boat, it could potentially be covered if it is considered maritime employment, but the claim lacks sufficient detail to confirm this. To make the claim valid, the policyholder should demonstrate that the injury occurred during employment directly related to maritime activities covered under the Jones Act, and that the injury was a result of employment duties on a vessel or maritime environment.",single_hop_specifc_query_synthesizer
1,"On April 10th, I noticed that a large section of my living room wall was damaged due to a leak from the upstairs bathroom. The leak started suddenly and caused the wall to become damp and stained. I called a plumber immediately, and they fixed the leak the same day. I believe this damage should be covered under my homeowner's insurance policy because it was an unexpected incident.","[the production of farm products and includes all\noperations reasonably necessary for such\nproduction that are performed:\n(a) at the request of the owner or renter of\nthose premises; and\n(b) in exchange for compensation in the form\nof money or goods.\nCustom farming does not mean operations\nperformed as part of an exchange of farming\nservices so long as no other compensation is\npaid.\n9. Declarations means the part of this policy titled\n“Homeowner’s Insurance Policy Declarations”. It\nsets out many of the individual facts related to\nyour policy including the dates, types, and\namounts, of the various coverages.\n10. Decorative fixture means wall covering, floor\ncovering, paint, and molding which is attached\nto the interior of:\n(a) your dwelling at the residence premises;\nor\n HO-4 (01-07)\n(b) other structures which are permanently\nattached to the residence premises, but not\nattached to your dwelling. If a structure is\nconnected to the dwelling by only a utility]","This claim is invalid based on the policy terms. The provided context indicates that the 'Homeowner’s Insurance Policy Declarations' include details about coverage, but it does not specify coverage for water damage caused by leaks or plumbing issues. Additionally, the policy mentions that coverage for damages resulting from gradual o

### Don't Throwaway?

In [130]:
from langchain_core.messages import ToolMessage

def extract_context(response):
    messages = response.get("messages", [])
    context = []
    for message in messages:
        if isinstance(message, ToolMessage):
            tool_context = []
            content = message.content
            documents = content.split("Document")[0:]
        
            for document in documents:
                if "page_content" in document:
                    page_content = document.split("page_content")[1].split("'")[1]
                    tool_context.append(page_content)
            if tool_context:
                context.append(f"{message.name}: {tool_context}")
        
    return context
            
    

## Upload Golden Dataset to Langsmith

In [127]:
from langsmith import Client

client = Client()

dataset_name = "Insurance Claim Data 3"

langsmith_dataset = client.create_dataset(
    dataset_name=dataset_name,
    description="Insurance Claim Data 3"
)

In [128]:
for data_row in claim_dataset.to_pandas().iterrows():
  client.create_example(
      inputs={
          "question": data_row[1]["user_input"]
      },
      outputs={
          "answer": data_row[1]["reference"]
      },
      metadata={
          "context": data_row[1]["reference_contexts"]
      },
      dataset_id=langsmith_dataset.id
  )

## Evaluations

#### Initial Prototype

In [131]:
import copy
evaluated_dataset = copy.deepcopy(claim_dataset)
for test_row in list(evaluated_dataset):
    result = claim_consultant_agent.invoke({"user_input" : test_row.eval_sample.user_input})
    context = extract_context(result)
    test_row.eval_sample.retrieved_contexts = context
    test_row.eval_sample.response = result["messages"][-1].content


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request

In [135]:
evaluated_dataset.to_pandas()

user_input  \
0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  On April 10th, I was injured while working on my boat at the marina, which I believe is covered under the Jones Act. I sustained a back injury after lifting heavy equipment, and I am now unable to work. I am requesting benefits for my injury.   
1                                                                                                                                                                                                                                                                                                                                                                                                                     On April 10th, I noticed that a large section of my living room wall was damaged due to a leak from the upstairs bathroom. The leak started suddenly and caused the wall to become damp and stained. I called a plumber immediately, and they fixed the leak the same day. I believe this damage should be covered under my homeowner's insurance policy because it was an unexpected incident.   
2                                                                                                                                                                                                                                                                     On April 10th, I noticed that my detached garage, which is connected to my property by a fence, was severely damaged during a storm. The roof was torn off, and the walls suffered significant structural damage. I am claiming the cost to repair or rebuild the garage, including the damage to the structure and roof. The garage is not attached to my main dwelling, but it is a permanent structure on my property. I would like to understand if this damage is covered under my HO-4 policy and how depreciation might affect my claim.   
3   I am a renter who recently experienced a minor fire in my apartment that caused some damage to the walls and flooring. The repair process involved replacing some wall coverings and floor coverings, which were decorative fixtures. Additionally, the fire caused some damage that reduced the overall value of my apartment, and I was unable to use certain parts of the property during repairs. I am now requesting compensation for the diminution of pre-loss value, the cost of replacing decorative fixtures, and the economic damages resulting from the loss of use of my apartment. I also want to ensure that my claim includes damages caused by the inability to match the repaired parts with undamaged adjacent parts, and I am concerned about whether my status as a renter affects coverage.   
4                                                                                        My house has been here for many years, and I think some damage happened over time because of how the land and the construction of the dwelling are. I noticed cracks in the walls and some parts of the land around my home seem to have shifted or settled, but I didn't see any sudden event. I believe this damage is covered because it relates to the construction and land, even though it developed slowly and I haven't had any recent storms or accidents. I want to file a claim for this damage, but I am not sure if the policy covers issues that happen gradually or over time, especially since I use the land and structures for storage and sometimes rent parts of the property for non-business purposes.   
5                    

In [136]:
from ragas import EvaluationDataset

evaluation_dataset = EvaluationDataset.from_pandas(evaluated_dataset.to_pandas())

In [137]:
from ragas.llms import LangchainLLMWrapper
evaluator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4.1-mini"))

In [138]:
from ragas.metrics import LLMContextRecall, Faithfulness, FactualCorrectness, ResponseRelevancy, ContextEntityRecall, NoiseSensitivity
from ragas import evaluate, RunConfig
from ragas import evaluate

custom_run_config = RunConfig(timeout=360)

result = evaluate(
    dataset=evaluation_dataset,
    metrics=[LLMContextRecall(), Faithfulness(), FactualCorrectness(), ResponseRelevancy(), ContextEntityRecall(), NoiseSensitivity()],
    llm=evaluator_llm,
    run_config=custom_run_config
)
result

Evaluating:   0%|          | 0/66 [00:00<?, ?it/s]INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
Evaluating:   2%|▏         | 1/66 [00:01<01:47,  1.65s/it]INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 20

{'context_recall': 0.5802, 'faithfulness': 0.3125, 'factual_correctness(mode=f1)': 0.5155, 'answer_relevancy': 0.5676, 'context_entity_recall': 0.0265, 'noise_sensitivity(mode=relevant)': 0.1366}

##### LangSmith

In [139]:
eval_llm = ChatOpenAI(model="gpt-4.1")

In [140]:
from langsmith.evaluation import LangChainStringEvaluator, evaluate

qa_evaluator = LangChainStringEvaluator("qa", config={"llm" : eval_llm})

labeled_helpfulness_evaluator = LangChainStringEvaluator(
    "labeled_criteria",
    config={
        "criteria": {
            "helpfulness": (
                "Is this submission helpful to the user,"
                " taking into account the correct reference answer?"
            )
        },
        "llm" : eval_llm
    },
    prepare_data=lambda run, example: {
        "prediction": run.outputs["output"],
        "reference": example.outputs["answer"],
        "input": example.inputs["question"],
    }
)

empathy_evaluator = LangChainStringEvaluator(
    "criteria",
    config={
        "criteria": {
            "empathy": "Is this response empathetic? Does it make the user feel like they are being heard?",
        },
        "llm" : eval_llm
    }
)

In [141]:
evaluate(
    claim_consultant_agent.invoke,
    data=dataset_name,
    evaluators=[
        qa_evaluator,
        labeled_helpfulness_evaluator,
        empathy_evaluator
    ],
    metadata={"revision_id": "basic_claim_consultant_agent"},
)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

View the evaluation results for experiment: 'frosty-driving-21' at:
https://smith.langchain.com/o/c2cfcbd8-d5df-509f-8f0e-973ec8ab5a6b/datasets/3ada1714-a65f-45ec-9b6f-7f9368ff9a1d/compare?selectedSessions=c489baf0-78d6-42cc-a847-4f850f0289c0




0it [00:00, ?it/s]ERROR:langsmith.evaluation._runner:Error running target function: 'user_input'
Traceback (most recent call last):
  File "/Users/terellbrown/terellcodes/claim-mate/api/.venv/lib/python3.12/site-packages/langsmith/evaluation/_runner.py", line 1924, in _forward
    fn(*args, langsmith_extra=langsmith_extra)
  File "/Users/terellbrown/terellcodes/claim-mate/api/.venv/lib/python3.12/site-packages/langgraph/pregel/main.py", line 3015, in invoke
    for chunk in self.stream(
                 ^^^^^^^^^^^^
  File "/Users/terellbrown/terellcodes/claim-mate/api/.venv/lib/python3.12/site-packages/langgraph/pregel/main.py", line 2642, in stream
    for _ in runner.tick(
             ^^^^^^^^^^^^
  File "/Users/terellbrown/terellcodes/claim-mate/api/.venv/lib/python3.12/site-packages/langgraph/pregel/_runner.py", line 162, in tick
    run_with_retry(
  File "/Users/terellbrown/terellcodes/claim-mate/api/.venv/lib/python3.12/site-packages/langgraph/pregel/_retry.py", line 42, in ru

,inputs.question,outputs.output,error,reference.answer,feedback.correctness,feedback.helpfulness,feedback.empathy,execution_time,example_id,id
0,"An elderly retiree notices a slow, persistent leak in their basement pipe that has been ongoing for several months. Over time, the water damage has caused mold growth and structural deterioration. The retiree files a claim, believing it is covered under their homeowner's policy. The policy states that coverage excludes damage caused by continuous or repeated seepage of water over weeks or months, but covers sudden and accidental discharge of water. Additionally, the policy applies only as excess insurance when other policies are involved, and it does not cover damages if the insured has received full compensation elsewhere. Considering these provisions, is the claim valid?",None,"KeyError('user_input')\n\nTraceback (most recent call last):\n File ""/Users/terellbrown/terellcodes/claim-mate/api/.venv/lib/python3.12/site-packages/langgraph/pregel/main.py"", line 3015, in invoke\n for chunk in self.stream(\n ^^^^^^^^^^^^\n File ""/Users/terellbrown/terellcodes/claim-mate/api/.venv/lib/python3.12/site-packages/langgraph/pregel/main.py"", line 2642, in stream\n for _ in runner.tick(\n ^^^^^^^^^^^^\n File ""/Users/terellbrown/terellcodes/claim-mate/api/.venv/lib/python3.12/site-packages/langgraph/pregel/_runner.py"", line 162, in tick\n run_with_retry(\n File ""/Users/terellbrown/terellcodes/claim-mate/api/.venv/lib/python3.12/site-packages/langgraph/pregel/_retry.py"", line 42, in run_with_retry\n return task.proc.invoke(task.input, config)\n ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^\n File ""/Users/terellbrown/terellcodes/claim-mate/api/.venv/lib/python3.12/site-packages/langgraph/_internal/_runnable.py"", line 657, in invoke\n input = context.run(step.invoke, input, config, **kwargs)\n ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^\n File ""/Users/terellbrown/terellcodes/claim-mate/api/.venv/lib/python3.12/site-packages/langgraph/_internal/_runnable.py"", line 401, in invoke\n ret = self.func(*args, **kwargs)\n ^^^^^^^^^^^^^^^^^^^^^^^^^^\n File ""/var/folders/r5/wsbp5_jj7lsfh6rxrw7p7lv00000gn/T/ipykernel_92811/3022482314.py"", line 40, in prepare_input\n messages.append(HumanMessage(content=state[""user_input""]))\n ~~~~~^^^^^^^^^^^^^^\nKeyError: 'user_input'\nDuring task with name 'prepare_input' and id '40cf7b75-8f90-c91c-f900-79bd77a97ebf'\n","This claim is invalid. The key issue is that the damage resulted from a slow, ongoing leak that developed over several months, which falls under the exclusion for damage caused by continuous or repeated seepage of water (<1-hop>). Since the damage was not caused by a sudden and accidental discharge, it is not covered under the policy's water damage provisions. Furthermore, the policy applies only as excess insurance and does not cover damages if the insured has received full compensation from another source (<2-hop>), but this is secondary to the primary coverage issue. To make the claim valid, the retiree would need to demonstrate that the water damage was caused by a sudden, accidental event rather than gradual seepage, or ensure that no other policies provide full coverage for the loss.",0,0,0,0.007415,a7283b16-b60f-4436-8b16-436aa461d7e3,256b206c-c430-4c6d-b661-bc7453f2fa78
1,"I had a fire in my garage last week, and my neighbor's shed, which is attached to my garage by a shared wall, was also damaged. I filed a claim expecting coverage for both structures. However, I also have another insurance policy that might cover part of the loss. Since the shed is attached to my garage, I believe my policy should cover the damage to both. I didn't realize I needed to check if the shed was considered attached or if other policies might apply. Please process my claim for the damage to my garage and the attached shed.",None,"KeyError('user_input')\n\nTraceback (most recent call last):\n File ""/Users/terellbrown/terellcodes/claim-mate/api/.venv/lib/python

#### Parent Document Prototype

In [146]:
parent_document_evaluated_dataset = copy.deepcopy(claim_dataset)
for test_row in list(parent_document_evaluated_dataset):
    result = parent_document_claim_consultant_agent.invoke({"user_input" : test_row.eval_sample.user_input})
    context = extract_context(result)
    test_row.eval_sample.retrieved_contexts = context
    test_row.eval_sample.response = result["messages"][-1].content

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request

In [147]:
parent_document_evaluated_dataset.to_pandas()

user_input  \
0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  On April 10th, I was injured while working on my boat at the marina, which I believe is covered under the Jones Act. I sustained a back injury after lifting heavy equipment, and I am now unable to work. I am requesting benefits for my injury.   
1                                                                                                                                                                                                                                                                                                                                                                                                                     On April 10th, I noticed that a large section of my living room wall was damaged due to a leak from the upstairs bathroom. The leak started suddenly and caused the wall to become damp and stained. I called a plumber immediately, and they fixed the leak the same day. I believe this damage should be covered under my homeowner's insurance policy because it was an unexpected incident.   
2                                                                                                                                                                                                                                                                     On April 10th, I noticed that my detached garage, which is connected to my property by a fence, was severely damaged during a storm. The roof was torn off, and the walls suffered significant structural damage. I am claiming the cost to repair or rebuild the garage, including the damage to the structure and roof. The garage is not attached to my main dwelling, but it is a permanent structure on my property. I would like to understand if this damage is covered under my HO-4 policy and how depreciation might affect my claim.   
3   I am a renter who recently experienced a minor fire in my apartment that caused some damage to the walls and flooring. The repair process involved replacing some wall coverings and floor coverings, which were decorative fixtures. Additionally, the fire caused some damage that reduced the overall value of my apartment, and I was unable to use certain parts of the property during repairs. I am now requesting compensation for the diminution of pre-loss value, the cost of replacing decorative fixtures, and the economic damages resulting from the loss of use of my apartment. I also want to ensure that my claim includes damages caused by the inability to match the repaired parts with undamaged adjacent parts, and I am concerned about whether my status as a renter affects coverage.   
4                                                                                        My house has been here for many years, and I think some damage happened over time because of how the land and the construction of the dwelling are. I noticed cracks in the walls and some parts of the land around my home seem to have shifted or settled, but I didn't see any sudden event. I believe this damage is covered because it relates to the construction and land, even though it developed slowly and I haven't had any recent storms or accidents. I want to file a claim for this damage, but I am not sure if the policy covers issues that happen gradually or over time, especially since I use the land and structures for storage and sometimes rent parts of the property for non-business purposes.   
5                    

In [148]:
parent_document_evaluation_dataset = EvaluationDataset.from_pandas(parent_document_evaluated_dataset.to_pandas())

In [149]:
from ragas.metrics import LLMContextRecall, Faithfulness, FactualCorrectness, ResponseRelevancy, ContextEntityRecall, NoiseSensitivity
from ragas import evaluate, RunConfig
from ragas import evaluate

custom_run_config = RunConfig(timeout=360)

result = evaluate(
    dataset=parent_document_evaluation_dataset,
    metrics=[LLMContextRecall(), Faithfulness(), FactualCorrectness(), ResponseRelevancy(), ContextEntityRecall(), NoiseSensitivity()],
    llm=evaluator_llm,
    run_config=custom_run_config
)
result

Evaluating:   0%|          | 0/66 [00:00<?, ?it/s]INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://ap

{'context_recall': 0.7788, 'faithfulness': 0.3511, 'factual_correctness(mode=f1)': 0.4845, 'answer_relevancy': 0.4049, 'context_entity_recall': 0.0673, 'noise_sensitivity(mode=relevant)': 0.1330}

In [151]:
from langsmith.evaluation import evaluate as langsmith_evaluate

langsmith_evaluate(
    parent_document_claim_consultant_agent.invoke,
    data=dataset_name,
    evaluators=[],
    metadata={"revision_id": "parent_document_claim_consultant_agent"},
)

View the evaluation results for experiment: 'virtual-distance-46' at:
https://smith.langchain.com/o/c2cfcbd8-d5df-509f-8f0e-973ec8ab5a6b/datasets/3ada1714-a65f-45ec-9b6f-7f9368ff9a1d/compare?selectedSessions=7e73ca37-a6b0-430d-a82a-d729b6b85a77




0it [00:00, ?it/s]ERROR:langsmith.evaluation._runner:Error running target function: 'user_input'
Traceback (most recent call last):
  File "/Users/terellbrown/terellcodes/claim-mate/api/.venv/lib/python3.12/site-packages/langsmith/evaluation/_runner.py", line 1924, in _forward
    fn(*args, langsmith_extra=langsmith_extra)
  File "/Users/terellbrown/terellcodes/claim-mate/api/.venv/lib/python3.12/site-packages/langgraph/pregel/main.py", line 3015, in invoke
    for chunk in self.stream(
                 ^^^^^^^^^^^^
  File "/Users/terellbrown/terellcodes/claim-mate/api/.venv/lib/python3.12/site-packages/langgraph/pregel/main.py", line 2642, in stream
    for _ in runner.tick(
             ^^^^^^^^^^^^
  File "/Users/terellbrown/terellcodes/claim-mate/api/.venv/lib/python3.12/site-packages/langgraph/pregel/_runner.py", line 162, in tick
    run_with_retry(
  File "/Users/terellbrown/terellcodes/claim-mate/api/.venv/lib/python3.12/site-packages/langgraph/pregel/_retry.py", line 42, in ru

,inputs.question,outputs.output,error,reference.answer,execution_time,example_id,id
0,"An elderly retiree notices a slow, persistent leak in their basement pipe that has been ongoing for several months. Over time, the water damage has caused mold growth and structural deterioration. The retiree files a claim, believing it is covered under their homeowner's policy. The policy states that coverage excludes damage caused by continuous or repeated seepage of water over weeks or months, but covers sudden and accidental discharge of water. Additionally, the policy applies only as excess insurance when other policies are involved, and it does not cover damages if the insured has received full compensation elsewhere. Considering these provisions, is the claim valid?",None,"KeyError('user_input')\n\nTraceback (most recent call last):\n File ""/Users/terellbrown/terellcodes/claim-mate/api/.venv/lib/python3.12/site-packages/langgraph/pregel/main.py"", line 3015, in invoke\n for chunk in self.stream(\n ^^^^^^^^^^^^\n File ""/Users/terellbrown/terellcodes/claim-mate/api/.venv/lib/python3.12/site-packages/langgraph/pregel/main.py"", line 2642, in stream\n for _ in runner.tick(\n ^^^^^^^^^^^^\n File ""/Users/terellbrown/terellcodes/claim-mate/api/.venv/lib/python3.12/site-packages/langgraph/pregel/_runner.py"", line 162, in tick\n run_with_retry(\n File ""/Users/terellbrown/terellcodes/claim-mate/api/.venv/lib/python3.12/site-packages/langgraph/pregel/_retry.py"", line 42, in run_with_retry\n return task.proc.invoke(task.input, config)\n ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^\n File ""/Users/terellbrown/terellcodes/claim-mate/api/.venv/lib/python3.12/site-packages/langgraph/_internal/_runnable.py"", line 657, in invoke\n input = context.run(step.invoke, input, config, **kwargs)\n ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^\n File ""/Users/terellbrown/terellcodes/claim-mate/api/.venv/lib/python3.12/site-packages/langgraph/_internal/_runnable.py"", line 401, in invoke\n ret = self.func(*args, **kwargs)\n ^^^^^^^^^^^^^^^^^^^^^^^^^^\n File ""/var/folders/r5/wsbp5_jj7lsfh6rxrw7p7lv00000gn/T/ipykernel_92811/3022482314.py"", line 40, in prepare_input\n messages.append(HumanMessage(content=state[""user_input""]))\n ~~~~~^^^^^^^^^^^^^^\nKeyError: 'user_input'\nDuring task with name 'prepare_input' and id 'b2d1d0e0-c4b8-16d8-23d8-547e0179ed74'\n","This claim is invalid. The key issue is that the damage resulted from a slow, ongoing leak that developed over several months, which falls under the exclusion for damage caused by continuous or repeated seepage of water (<1-hop>). Since the damage was not caused by a sudden and accidental discharge, it is not covered under the policy's water damage provisions. Furthermore, the policy applies only as excess insurance and does not cover damages if the insured has received full compensation from another source (<2-hop>), but this is secondary to the primary coverage issue. To make the claim valid, the retiree would need to demonstrate that the water damage was caused by a sudden, accidental event rather than gradual seepage, or ensure that no other policies provide full coverage for the loss.",0.007903,a7283b16-b60f-4436-8b16-436aa461d7e3,b57f3fd5-16b1-42a3-b65c-2c836ee39a42
1,"I had a fire in my garage last week, and my neighbor's shed, which is attached to my garage by a shared wall, was also damaged. I filed a claim expecting coverage for both structures. However, I also have another insurance policy that might cover part of the loss. Since the shed is attached to my garage, I believe my policy should cover the damage to both. I didn't realize I needed to check if the shed was considered attached or if other policies might apply. Please process my claim for the damage to my garage and the attached shed.",None,"KeyError('user_input')\n\nTraceback (most recent call last):\n File ""/Users/terellbrown/terellcodes/claim-mate/api/.venv/lib/python3.12/site-packages/langgraph/pregel/main.py"", line 3015, in invo

#### Contextual Compression Prototype

In [142]:
advanced_evaluated_dataset = copy.deepcopy(claim_dataset)
for test_row in list(advanced_evaluated_dataset):
    result = advanced_claim_consultant_agent.invoke({"user_input" : test_row.eval_sample.user_input})
    context = extract_context(result)
    test_row.eval_sample.retrieved_contexts = context
    test_row.eval_sample.response = result["messages"][-1].content


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request

In [143]:
advanced_evaluated_dataset.to_pandas()

user_input  \
0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  On April 10th, I was injured while working on my boat at the marina, which I believe is covered under the Jones Act. I sustained a back injury after lifting heavy equipment, and I am now unable to work. I am requesting benefits for my injury.   
1                                                                                                                                                                                                                                                                                                                                                                                                                     On April 10th, I noticed that a large section of my living room wall was damaged due to a leak from the upstairs bathroom. The leak started suddenly and caused the wall to become damp and stained. I called a plumber immediately, and they fixed the leak the same day. I believe this damage should be covered under my homeowner's insurance policy because it was an unexpected incident.   
2                                                                                                                                                                                                                                                                     On April 10th, I noticed that my detached garage, which is connected to my property by a fence, was severely damaged during a storm. The roof was torn off, and the walls suffered significant structural damage. I am claiming the cost to repair or rebuild the garage, including the damage to the structure and roof. The garage is not attached to my main dwelling, but it is a permanent structure on my property. I would like to understand if this damage is covered under my HO-4 policy and how depreciation might affect my claim.   
3   I am a renter who recently experienced a minor fire in my apartment that caused some damage to the walls and flooring. The repair process involved replacing some wall coverings and floor coverings, which were decorative fixtures. Additionally, the fire caused some damage that reduced the overall value of my apartment, and I was unable to use certain parts of the property during repairs. I am now requesting compensation for the diminution of pre-loss value, the cost of replacing decorative fixtures, and the economic damages resulting from the loss of use of my apartment. I also want to ensure that my claim includes damages caused by the inability to match the repaired parts with undamaged adjacent parts, and I am concerned about whether my status as a renter affects coverage.   
4                                                                                        My house has been here for many years, and I think some damage happened over time because of how the land and the construction of the dwelling are. I noticed cracks in the walls and some parts of the land around my home seem to have shifted or settled, but I didn't see any sudden event. I believe this damage is covered because it relates to the construction and land, even though it developed slowly and I haven't had any recent storms or accidents. I want to file a claim for this damage, but I am not sure if the policy covers issues that happen gradually or over time, especially since I use the land and structures for storage and sometimes rent parts of the property for non-business purposes.   
5                    

In [144]:
advanced_evaluation_dataset = EvaluationDataset.from_pandas(advanced_evaluated_dataset.to_pandas())

In [145]:
from ragas.metrics import LLMContextRecall, Faithfulness, FactualCorrectness, ResponseRelevancy, ContextEntityRecall, NoiseSensitivity
from ragas import evaluate, RunConfig
from ragas import evaluate

custom_run_config = RunConfig(timeout=360)

result = evaluate(
    dataset=advanced_evaluation_dataset,
    metrics=[LLMContextRecall(), Faithfulness(), FactualCorrectness(), ResponseRelevancy(), ContextEntityRecall(), NoiseSensitivity()],
    llm=evaluator_llm,
    run_config=custom_run_config
)
result

Evaluating:   0%|          | 0/66 [00:00<?, ?it/s]INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
Evaluating:   2%|▏         | 1/66 [00:01<01:55,  1.77s/it]INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 20

{'context_recall': 0.7879, 'faithfulness': 0.3572, 'factual_correctness(mode=f1)': 0.4591, 'answer_relevancy': 0.4057, 'context_entity_recall': 0.0866, 'noise_sensitivity(mode=relevant)': 0.0925}

In [ ]:
from langsmith.evaluation import evaluate as langsmith_evaluate

langsmith_evaluate(
    advanced_claim_consultant_agent.invoke,
    data=dataset_name,
    evaluators=[],
    metadata={"revision_id": "advanced_claim_consultant_agent"},
)